在 Python 中，优雅地共享“全局”状态，尤其是在递归等场景下，通常**不推荐使用真正的 `global` 关键字**。真正的全局变量会污染命名空间，并使代码难以理解和测试。

你提供的第一种方法（使用类实例变量 `self.balance`）本身就是一种非常优雅和常见的面向对象解决方案。第二种方法失败了，因为它涉及 Python 的一个核心概念：**参数传递机制**。

下面我将分析你的代码，并介绍几种更优雅的共享状态的方式。

**代码参考 class037/code04_balance_binary_tree**

-----

### \#\# 代码分析：为什么第一种可行，第二种不行？

#### 你的第一种解法 (可行 ✅)

```python
class Solution:
    def isBalanced(self, root: Optional[TreeNode]) -> bool:
        # self.balance 是一个实例变量，属于 Solution 这个对象
        self.balance = True
        if root:
            # 每次递归调用 self.verify_height 时，
            # 内部的 self 都指向同一个 Solution 实例
            self.verify_height(root)
        return self.balance

    def verify_height(self, root: TreeNode) -> int:
        if not self.balance or root is None:
            return 0

        left_height = self.verify_height(root.left)
        right_height = self.verify_height(root.right)

        if abs(left_height - right_height) > 1:
            # 这里修改的是 self 指向的那个对象的 balance 属性
            # 这个修改在所有递归调用中都是可见的
            self.balance = False

        return max(left_height, right_height) + 1
```

这种方法之所以有效，是因为 `self.balance` 是**实例对象的状态**。在整个 `isBalanced` 调用过程中，`self` 始终是同一个 `Solution` 类的实例。因此，任何一次递归调用对 `self.balance` 的修改，都会改变这个共享实例的状态，从而被其他所有递归调用立即“看到”。这是一种优秀的、利用对象封装状态的解决方案。

#### 你的第二种解法 (不可行 ❌)

```python
class Solution:
    def isBalanced(self, root: Optional[TreeNode]) -> bool:
        # balance 是 isBalanced 方法内部的一个局部变量
        balance = True
        if root:
            # 你传递的是 balance 的值 (True)，而不是变量本身
            self.verify_height(root, global_balance=balance)
        # 这里的 balance 从未被修改过，永远是 True
        return balance

    def verify_height(self, root: TreeNode, global_balance: bool) -> int:
        if not global_balance or root is None:
            return 0

        left_height = self.verify_height(root.left, global_balance)
        right_height = self.verify_height(root.right, global_balance)

        if abs(left_height - right_height) > 1:
            # 关键错误点：这里只是将 *局部参数* global_balance
            # 重新指向了 False 这个值。
            # 它完全没有影响到 isBalanced 方法中的那个局部变量 balance。
            global_balance = False

        return max(left_height, right_height) + 1
```

这种方法失败的原因在于 Python 的参数传递方式。当你传递 `balance` (一个布尔值) 时：

  * **布尔值 (Booleans)、整数 (Integers)、字符串 (Strings) 等是不可变类型 (Immutable)。**
  * 当你执行 `global_balance = False` 时，你并没有改变原始 `balance` 变量。你只是让 `verify_height` 函数作用域内的局部名称 `global_balance` 指向了一个新的对象 `False`。
  * 一旦 `verify_height` 函数返回，这个局部的修改就丢失了。外部的 `balance` 变量从未被触及。

-----

### \#\# 优雅共享状态的几种方式

除了你已经成功的“实例变量”法，这里还有几种更 Pythonic、更通用的方法。

#### 方式一：通过返回值传递状态 (函数式风格)

这是最推荐的方法之一，因为它避免了“副作用”（即在函数内部修改外部状态）。函数只做一件事：接收输入，返回输出。

我们将让辅助函数同时返回两个信息：**子树是否平衡**和**子树的高度**。

```python
class Solution:
    def isBalanced(self, root: Optional[TreeNode]) -> bool:
        # 调用辅助函数，我们只关心第一个返回值 (is_balanced)
        return self._verify(root)[0]

    def _verify(self, node: Optional[TreeNode]) -> (bool, int):
        """
        返回一个元组: (子树是否平衡, 子树的高度)
        """
        if not node:
            return (True, 0)

        # 递归获取左右子树的信息
        is_left_balanced, left_height = self._verify(node.left)
        is_right_balanced, right_height = self._verify(node.right)

        # 判断当前节点是否平衡
        # 条件：1. 左子树平衡 2. 右子树平衡 3. 左右子树高度差不超过1
        node_is_balanced = (is_left_balanced and
                            is_right_balanced and
                            abs(left_height - right_height) <= 1)

        current_height = max(left_height, right_height) + 1

        return (node_is_balanced, current_height)
```

**优点**: 代码清晰，没有隐藏的状态修改，更容易测试和推理。

#### 方式二：使用 `nonlocal` 关键字 (闭包)

如果你喜欢将辅助函数定义在主函数内部，`nonlocal` 是一个绝佳的选择。它允许内部函数修改外部（但非全局）函数的变量。

```python
class Solution:
    def isBalanced(self, root: Optional[TreeNode]) -> bool:
        # 定义一个外部函数作用域的变量
        is_balanced = True

        def get_height(node: Optional[TreeNode]) -> int:
            nonlocal is_balanced
            # 如果已经发现不平衡，后续的计算可以提前终止，进行剪枝
            if not node or not is_balanced:
                return 0

            left_height = get_height(node.left)
            right_height = get_height(node.right)

            if abs(left_height - right_height) > 1:
                # 修改外部函数的 is_balanced 变量
                is_balanced = False

            return max(left_height, right_height) + 1

        get_height(root)
        return is_balanced
```

**优点**: 逻辑内聚，辅助函数 `get_height` 对外不可见，避免了污染类的命名空间。

#### 方式三：传递可变类型对象 (如列表或字典)

这个方法利用了 Python 中“万物皆对象”的特性。虽然不能直接修改传入的布尔值，但可以修改传入的\*\*可变对象（Mutable Object）\*\*的内容。

```python
class Solution:
    def isBalanced(self, root: Optional[TreeNode]) -> bool:
        # 使用一个列表来包装布尔值，列表是可变类型
        balance_tracker = [True]
        self.get_height(root, balance_tracker)
        return balance_tracker[0]

    def get_height(self, node: Optional[TreeNode], tracker: list) -> int:
        if not node or not tracker[0]:
            return 0

        left = self.get_height(node.left, tracker)
        right = self.get_height(node.right, tracker)

        if abs(left - right) > 1:
            # 修改列表中的元素，这个修改在所有调用中都可见
            tracker[0] = False

        return max(left, right) + 1
```

**优点**: 这是一种巧妙的技巧，可以让你在不使用 `nonlocal` 或实例变量的情况下跨函数调用修改状态。

### \#\# 总结

| 方式 | 优点 | 缺点 | 适用场景 |
| :--- | :--- | :--- | :--- |
| **1. 实例变量 (你的第一种)** | 面向对象风格，逻辑清晰，状态与对象绑定。 | 需要将逻辑放在一个类中。 | 解决类方法中的递归问题，如 LeetCode。 |
| **2. 返回多值 (函数式)** | 无副作用，纯函数，逻辑清晰，易于测试。 | 可能会增加返回值的复杂度。 | 强烈推荐的通用方法，尤其适合复杂递归。 |
| **3. `nonlocal` (闭包)** | 封装性好，辅助逻辑不暴露。 | 只能用于嵌套函数结构。 | 当辅助函数只为某个主函数服务时。 |
| **4. 传递可变对象** | 技巧性强，无需类或闭包。 | 代码意图可能不如前几种清晰。 | 在无法使用其他方法时的灵活备选。 |

对于你提出的问题，**方法1（实例变量）** 和 **方法2（返回多值）** 通常被认为是最优雅、最主流的解决方案。